In [1]:
import numpy as np
import matplotlib.pyplot as plt

import joblib

import random
from deap import base
from deap import creator
from deap import tools

from deap import algorithms
from time import time
import multiprocessing

import array
import random
import json

import os

import pandas as pd

In [2]:
matrix_np = joblib.load('matrix_np.pkl')

In [3]:
#reduced_matrix_np = joblib.load('reduced_matrix_np.pkl')

# Parameters

In [4]:
matrix_algorithm = matrix_np
NUM_CLUSTERS = 7

# Distance to cluster function

In [5]:
def auxiliarFunction(documents, dimensions, number_topics):

    counter_docs = [0 for x in range(0,number_topics)]
    
    distance_to_cluster_2 = [0 for x in range(0,number_topics)] 
    
    for document in documents: 
        distance_to_cluster = []   
        counter = 0
        for dim in range(0,number_topics): 
            distance_to_cluster.append(np.linalg.norm(document-dimensions[counter:counter+7]))
            counter = counter + 7
            
        best_distance_index = distance_to_cluster.index(min(distance_to_cluster)) 
        
        counter_docs[best_distance_index] += 1
    
        distance_to_cluster_2[best_distance_index] += pow(distance_to_cluster[best_distance_index], 2)
        
     
    return counter_docs, distance_to_cluster_2

In [6]:
def fobjQuadraticError(k):

    num_topics = int((len(k))/7)
    
    # 1º Number of documents to each cluster
    counter_documents, euclidean_dist_2 = auxiliarFunction(documents = matrix_algorithm, dimensions= k, number_topics = num_topics)
    
    bool_check = 0 in counter_documents    

   # 2º Fitness fuction
    if(bool_check):
        total_value = 5 # We give a high value to those clusters who do not have documents
        
    else:
        value1 = 0
        for i in range(0,num_topics): 
            value2 = euclidean_dist_2[i] 
            value1 += ((1/counter_documents[i])*value2)

        total_value = (value1 / num_topics) 

    return (total_value),

# Genetic Algorithm

## Folders

In [7]:
path = "./results/fobjQuadraticError"
try:
    if not os.path.isdir(path):
        os.makedirs(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Successfully created the directory ./results/fobjQuadraticError 


## Genetic operators

### Initial Population

In [8]:
def load_individuals(creator,n):
    individuals = []
    ones = [0 for x in range(0,49)]
    j=0
    for i in range(49):
        if (i % 7 == 0):
            ones[i+j] = 1
            j = j+1
            
    individual = creator(ones)
    individuals.append(individual)
    
    for l in range(int(n*0.1)-1):
        one = [0 for x in range(0,49)]
        m1 = [0,1,2,3,4,5,6]
        random.shuffle(m1)
        for j in range (7):
            for i in range(7):
                one[7*j+i] = ones[7*m1[j]+i]
        individual = creator(one)
        individuals.append(individual)
        
    
    Rfactor = [0 for x in range(0,3)]
    for i in range(3):
        Rfactor[i] = 1/4*(i+1)
    
    for l in range(int(n*0.9/3)):
        two = [0 for x in range(0,49)]
        ind1 = individuals[random.randrange(0, int(n*0.1))]
        ind2 = individuals[random.randrange(0, int(n*0.1))]
        for t in range(3):
            two = [0 for x in range(0,49)]
            for j in range(7):
                for i in range(7):
                    two[7*j+i] = ind1[7*j+i]*Rfactor[t]+ind2[7*j+i]*(1-Rfactor[t])
                
            individual = creator(two)
            individuals.append(individual)
        
    return individuals

### Mutation operators

#### Global mutation operator

In [9]:
def customMutate(ind,b_ind,g,m_prob):
    t = random.randrange(0, 10)
    
    if t>4 and g >1:
        new_clusters = redist2(b_ind)
        ind[0:49] = new_clusters[0:49]
    else: #first mutation operator
        for j in range (7):
            if random.random() < m_prob:
                m1 = random.randrange(0, 7)
                m2 = random.randrange(0, 7)
                auxNum = ind[7*j+m1]
                ind[7*j+m1] = ind[7*j+m2]
                ind[7*j+m2] = auxNum        
        
    return ind,

In [10]:
def isone(k):
    sumat = 0
    for i in range(7):
        sumat = sumat + k[i]
    return sumat

In [11]:
def checkbounds(k):
    for i in range(7):
        if k[i]>1:
            k[i]=1
        if k[i]<0:
            k[i]=0
    return k

In [12]:
def redist(b_ind):
    documents = matrix_algorithm 
    new_clusters = [0 for x in range(0,49)]
    counter = 0
    for dim in range(0,7): 
        min_dist_doc = [0 for x in range(0,7)]
        max_dist_doc = [0 for x in range(0,7)]
        min_dist = 100000000
        max_dist = 0
        for document in documents: 
                dist = np.linalg.norm(document-b_ind[counter:counter+7])
                if dist<min_dist:
                    min_dist = dist
                    min_dist_doc = document
                if dist>max_dist:
                    max_dist = dist
                    max_dist_doc = document
                    
        new_clusters[counter:counter+7] = (min_dist_doc+max_dist_doc)/2
        counter = counter + 7
                 
    return new_clusters

#### 2nd Mutation Operator

In [13]:
def redist2(b_ind):
    documents = matrix_algorithm 
    new_clusters = [0 for x in range(0,49)]
    diff = [0 for x in range(0,49)]
    counter = 0
    Rf = random.random()
    for dim in range(0,7): 
        min_dist_doc = [0 for x in range(0,7)]
        max_dist_doc = [0 for x in range(0,7)]
        
        min_dist = 100000000
        max_dist = 0
        for document in documents: 
                dist = np.linalg.norm(document-b_ind[counter:counter+7])
                if dist<min_dist:
                    min_dist = dist
                    min_dist_doc = document
                if dist>max_dist:
                    max_dist = dist
                    max_dist_doc = document
 

        diff[counter:counter+7] = (min_dist_doc+max_dist_doc)/2 - b_ind[counter:counter+7] 

        for j in range(counter,counter+7):
            new_clusters[j] = b_ind[j]+Rf*diff[j]
        
        sumat = isone(new_clusters[counter:counter+7])    
        while sumat<0.99999999999999 or sumat>1.0000000000000:
            new_clusters[counter:counter+7] = new_clusters[counter:counter+7]+ (1-sumat)/7
            new_clusters = checkbounds(new_clusters)
            sumat = isone(new_clusters[counter:counter+7]) 
            
        counter = counter + 7
        
    return new_clusters

### Define evaluation function

In [14]:
def evalfobj(individual):
    return fobjQuadraticError(individual)

### Crossover operator

In [15]:
def customMate(ind1, ind2):
    aux1 = ind1
    aux2 = ind2
      
    Rfactor = [0 for x in range(0,2)]
    for i in range(2):
        Rfactor[i] = random.random()
    for i in range(49):
        ind1[i] = aux1[i]*Rfactor[0]+aux2[i]*(1-Rfactor[0])
        ind2[i] = aux1[i]*Rfactor[1]+aux2[i]*(1-Rfactor[1])

    return (ind1,ind2)

## Genetic process and save results 

In [16]:
if __name__ == "__main__":
    
    CLUSTER_DIM = 7 * NUM_CLUSTERS

    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    
    toolbox.register("population", load_individuals,  creator.Individual)


    #toolbox.register("attr_flt", random.uniform, 0, 1)
    #toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_flt,n=CLUSTER_DIM)

    #toolbox.register("population", tools.initRepeat, list, toolbox.individual)


    toolbox.register("mate", customMate)
    toolbox.register("mutate", customMutate)

    toolbox.register("select", tools.selTournament, tournsize=5)
    
    toolbox.register("evaluate", evalfobj)
    
    pool = multiprocessing.Pool(processes = 25)
    toolbox.register('map', pool.map)
    
    numberInd = 100
    
    # MAIN GENETIC ALGORITHM
    random.seed(169)
    min_iterations = []
    max_iterations = []
    avg_iterations = []
    std_iterations = []
    ind_iterations = []
    
    CXPB, MUTPB = 0.8, 0.4
    m_prob = 0.35
    
    pop = toolbox.population(n=100)
    fitnesses = list(toolbox.map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit 
        
    fits = [ind.fitness.values[0] for ind in pop] 
    best_indiv = []
    g = 0
    # Begin the evolution
    while g < 10000:

       # start_time = time()
    
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)

        offspring = toolbox.select(pop, len(pop))
    
        offspring = list(map(toolbox.clone, offspring))
        

        for child1, child2 in zip(offspring[::2], offspring[1::2]): #DE los padres, creas 2 hijos con mate
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values 
                del child2.fitness.values 
                
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant,best_indiv,g,m_prob)
                del mutant.fitness.values  

        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
            
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
            

        pop[:] = offspring

        
        fits = [ind.fitness.values[0] for ind in pop]
        
        fit_maxi = 100000000000000000000
        individ = []
        for ind in pop:
            if (ind.fitness.values[0]<fit_maxi):
                fit_maxi = ind.fitness.values[0]
                individ = ind
        best_indiv = individ

            
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        print("  Min %s" % min(fits))
        
  
        
        min_iterations.append(min(fits))
        max_iterations.append(max(fits))
        avg_iterations.append(mean)
        std_iterations.append(std)
        ind_iterations.append(individ)
        
        if (g % 100 == 0):
            print(best_indiv)
            path = "./results/fobjQuadraticError/iteration{}".format(g)
            try:
                if not os.path.isdir(path):
                    os.makedirs(path)
            except OSError:
                print ("Creation of the directory %s failed" % path)
            else:
                print ("Successfully created the directory %s " % path)
        
            print("-----------------------------------------------------------------------------------------")
            print("Guardamos valores")
            joblib.dump(min_iterations, "results/fobjQuadraticError/iteration{}/best_gen_fitness_iter{}.pkl".format(g, g))
            joblib.dump(max_iterations, "results/fobjQuadraticError/iteration{}/worst_gen_fitness_iter{}.pkl".format(g, g))
            joblib.dump(avg_iterations, "results/fobjQuadraticError/iteration{}/mean_gen_fitness_iter{}.pkl".format(g, g))
            joblib.dump(std_iterations, "results/fobjQuadraticError/iteration{}/std_gen_fitness_iter{}.pkl".format(g, g))
            joblib.dump(ind_iterations, "results/fobjQuadraticError/iteration{}/best_gen_individuals_iter{}.pkl".format(g, g))
            joblib.dump(pop[:], "results/fobjQuadraticError/iteration{}/last_pop{}.pkl".format(g, g))

            print('-----------------------------------------------------------------------------------------')

       # print("--- {} s seconds to perform iteration number: {} ---".format((time() - start_time), g))
            
    #best = pop[np.argmin([toolbox.evaluate(x) for x in pop])]
    final_pop = pop[:]
    
    joblib.dump(min_iterations, "results/fobjQuadraticError/best_gen_fitness_iter{}.pkl".format(10000))
    joblib.dump(max_iterations, "results/fobjQuadraticError/worst_gen_fitness_iter{}.pkl".format(10000))
    joblib.dump(avg_iterations, "results/fobjQuadraticError/mean_gen_fitness_iter{}.pkl".format(10000))
    joblib.dump(std_iterations, "results/fobjQuadraticError/std_gen_fitness_iter{}.pkl".format(10000))
    joblib.dump(ind_iterations, "results/fobjQuadraticError/best_gen_individuals_iter{}.pkl".format(10000))
    joblib.dump(final_pop, "results/fobjQuadraticError/final_pop_iter{}.pkl".format(10000))


-- Generation 1 --
  Min 0.1346903183943112
-- Generation 2 --
  Min 0.12185528962925865
-- Generation 3 --
  Min 0.11470958579317517
-- Generation 4 --
  Min 0.1118979221707427
-- Generation 5 --
  Min 0.1117025612101329
-- Generation 6 --
  Min 0.11064618675613326
-- Generation 7 --
  Min 0.10696428664954173
-- Generation 8 --
  Min 0.10663757277833395
-- Generation 9 --
  Min 0.1045436586682784
-- Generation 10 --
  Min 0.10305622450549523
-- Generation 11 --
  Min 0.10238057466784725
-- Generation 12 --
  Min 0.10183091768681254
-- Generation 13 --
  Min 0.10153100620153853
-- Generation 14 --
  Min 0.10073895366885674
-- Generation 15 --
  Min 0.10037757624538958
-- Generation 16 --
  Min 0.0996671337210659
-- Generation 17 --
  Min 0.09877660874400021
-- Generation 18 --
  Min 0.09798049498758661
-- Generation 19 --
  Min 0.09765881426679499
-- Generation 20 --
  Min 0.09717572570939315
-- Generation 21 --
  Min 0.09717572570939315
-- Generation 22 --
  Min 0.09716099747035227
--

  Min 0.0946162458832898
-- Generation 152 --
  Min 0.0946162458832898
-- Generation 153 --
  Min 0.0946162458832898
-- Generation 154 --
  Min 0.0946162458832898
-- Generation 155 --
  Min 0.0946162458832898
-- Generation 156 --
  Min 0.0946162459597374
-- Generation 157 --
  Min 0.0946162459597374
-- Generation 158 --
  Min 0.09461608498696525
-- Generation 159 --
  Min 0.0946156380964536
-- Generation 160 --
  Min 0.09461470346198944
-- Generation 161 --
  Min 0.09461467244638343
-- Generation 162 --
  Min 0.09461467244638343
-- Generation 163 --
  Min 0.094614537482571
-- Generation 164 --
  Min 0.09461453025976606
-- Generation 165 --
  Min 0.09461450294135705
-- Generation 166 --
  Min 0.09461450294135705
-- Generation 167 --
  Min 0.09461447480978905
-- Generation 168 --
  Min 0.09461447480978905
-- Generation 169 --
  Min 0.09461447384881018
-- Generation 170 --
  Min 0.09461447384881018
-- Generation 171 --
  Min 0.09461447384881018
-- Generation 172 --
  Min 0.094614474351600

  Min 0.09455998031433631
-- Generation 300 --
  Min 0.09455998031433631
[0.10191514297170995, 0.07242181952244539, 0.060230195919730486, 0.4384599801427319, 0.10973278812744997, 0.11293767597706486, 0.10430239733886726, 0.49254621910619123, 0.08851541878643443, 0.08378401480162609, 0.0901696566421544, 0.08417871955514197, 0.06919053534402227, 0.09161543576442943, 0.07938699991034837, 0.0664684499928928, 0.06216629309279777, 0.06304231998161537, 0.06898824440732663, 0.06187232079140523, 0.598075371823614, 0.0879550657372544, 0.4342206668446561, 0.1025493013424725, 0.08229462492452377, 0.10064570685078081, 0.0876773921318553, 0.10465724216845726, 0.08081370071012281, 0.14882268578586538, 0.08404373076421387, 0.08404684952288907, 0.08088777713268877, 0.43837186193989297, 0.08301339414432718, 0.09987441640516034, 0.09310622466033208, 0.085816835183636, 0.08620475554628326, 0.433873506451661, 0.08181761369510238, 0.11930664805782498, 0.09839757899421975, 0.09357774291916027, 0.433469279049

  Min 0.09425602998514475
-- Generation 421 --
  Min 0.09425602998514475
-- Generation 422 --
  Min 0.09425602998514475
-- Generation 423 --
  Min 0.09425603008797886
-- Generation 424 --
  Min 0.09425349677368955
-- Generation 425 --
  Min 0.09425342665862564
-- Generation 426 --
  Min 0.09425342517824686
-- Generation 427 --
  Min 0.09425342064847865
-- Generation 428 --
  Min 0.0942533540308413
-- Generation 429 --
  Min 0.0942533822360994
-- Generation 430 --
  Min 0.09425335385649349
-- Generation 431 --
  Min 0.09425335385660545
-- Generation 432 --
  Min 0.09425335385576773
-- Generation 433 --
  Min 0.09425335385581672
-- Generation 434 --
  Min 0.09425335385598059
-- Generation 435 --
  Min 0.09425335385591334
-- Generation 436 --
  Min 0.09425295132424202
-- Generation 437 --
  Min 0.09425189198655262
-- Generation 438 --
  Min 0.09425177291438355
-- Generation 439 --
  Min 0.09425168888099668
-- Generation 440 --
  Min 0.09425165232317712
-- Generation 441 --
  Min 0.0942516

Process ForkPoolWorker-18:
Process ForkPoolWorker-2:
Process ForkPoolWorker-21:
Process ForkPoolWorker-22:
Process ForkPoolWorker-3:
Process ForkPoolWorker-7:
Process ForkPoolWorker-12:
Process ForkPoolWorker-11:
Process ForkPoolWorker-16:
Process ForkPoolWorker-25:
Process ForkPoolWorker-1:
Process ForkPoolWorker-8:
Process ForkPoolWorker-6:
Process ForkPoolWorker-23:
Process ForkPoolWorker-14:
Process ForkPoolWorker-20:
Process ForkPoolWorker-15:
Process ForkPoolWorker-24:
Process ForkPoolWorker-13:
Process ForkPoolWorker-19:
Process ForkPoolWorker-17:
Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
T